In [5]:
import numpy as np
import pandas as pd
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print ("Libraries imported!")

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


bran

In [6]:
address = 'Dallas, Texas'

geolocator = Nominatim(user_agent='dallas_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Dallas is {}, {}.'.format(latitude, longitude))

The geographical coordinate of Dallas is 32.7762719, -96.7968559.


In [7]:
CLIENT_ID = 'ZL0RHWZXHZMSSZV21UG0554KNVUGX0GOEM55SSK4LCDN42IF'
CLIENT_SECRET = 'EZZKAHSGLKN1W05BGJYZD51Z33ZE5PHQVSJVH4OSP14SYKJV'
VERSION = '20180605'
LIMIT = 100
print('Your credentials:')
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID:ZL0RHWZXHZMSSZV21UG0554KNVUGX0GOEM55SSK4LCDN42IF
CLIENT SECRET:EZZKAHSGLKN1W05BGJYZD51Z33ZE5PHQVSJVH4OSP14SYKJV


In [9]:
search_query = 'Restaurant'
radius = 50000
print(search_query + '... OK')

Restaurant... OK


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ZL0RHWZXHZMSSZV21UG0554KNVUGX0GOEM55SSK4LCDN42IF&client_secret=EZZKAHSGLKN1W05BGJYZD51Z33ZE5PHQVSJVH4OSP14SYKJV&ll=32.7762719,-96.7968559&v=20180605&query=Restaurant&radius=50000&limit=100'

In [11]:
results = requests.get(url).json()

In [13]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,41311c80f964a520320d1fe3,1304 Elm St,...,"[1304 Elm St (Corner Elm & Field), Dallas, TX ...","[{'label': 'display', 'lat': 32.78096829549494...",32.780968,-96.801241,NaN,75202,TX,Enchilada's Restaurant,v-1585588012,509837611
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b15f41cf964a52002b623e3,1301 Main St,...,"[1301 Main St (at Field St.), Dallas, TX 75202...","[{'label': 'display', 'lat': 32.78037110712935...",32.780371,-96.801106,NaN,75202,TX,Ravenna Urban Italian Restaurant,v-1585588012,NaN
2,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f442266d4f2bdcc71def513,1717 N Akard St,...,"[1717 N Akard St (in The Fairmont Dallas), Dal...","[{'label': 'display', 'lat': 32.78597407382344...",32.785974,-96.801532,NaN,75201,TX,Pyramid Restaurant & Bar,v-1585588012,NaN
3,"[{'id': '4bf58dd8d48988d1d5941735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b738b87f964a5206cb32de3,650 N Pearl St,...,"[650 N Pearl St (in Marriott Hotel), Dallas, T...","[{'label': 'display', 'lat': 32.78691622920842...",32.786916,-96.795694,NaN,75201,TX,650 North Restaurant,v-1585588012,NaN
4,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b1a8a2af964a520e0eb23e3,3917 Cedar Springs Rd,...,"[3917 Cedar Springs Rd (at Throckmorton St.), ...","[{'label': 'display', 'lat': 32.81077020211321...",32.810770,-96.811049,NaN,75219,TX,Panda's Restaurant & Bar,v-1585588012,NaN


In [17]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Enchilada's Restaurant,Mexican Restaurant,1304 Elm St,US,Dallas,United States,Corner Elm & Field,664,"[1304 Elm St (Corner Elm & Field), Dallas, TX ...","[{'label': 'display', 'lat': 32.78096829549494...",32.780968,-96.801241,NaN,75202,TX,41311c80f964a520320d1fe3
1,Ravenna Urban Italian Restaurant,Italian Restaurant,1301 Main St,US,Dallas,United States,at Field St.,605,"[1301 Main St (at Field St.), Dallas, TX 75202...","[{'label': 'display', 'lat': 32.78037110712935...",32.780371,-96.801106,NaN,75202,TX,4b15f41cf964a52002b623e3
2,Pyramid Restaurant & Bar,American Restaurant,1717 N Akard St,US,Dallas,United States,in The Fairmont Dallas,1165,"[1717 N Akard St (in The Fairmont Dallas), Dal...","[{'label': 'display', 'lat': 32.78597407382344...",32.785974,-96.801532,NaN,75201,TX,4f442266d4f2bdcc71def513
3,650 North Restaurant,Hotel Bar,650 N Pearl St,US,Dallas,United States,in Marriott Hotel,1189,"[650 N Pearl St (in Marriott Hotel), Dallas, T...","[{'label': 'display', 'lat': 32.78691622920842...",32.786916,-96.795694,NaN,75201,TX,4b738b87f964a5206cb32de3
4,Panda's Restaurant & Bar,Chinese Restaurant,3917 Cedar Springs Rd,US,Dallas,United States,at Throckmorton St.,4063,"[3917 Cedar Springs Rd (at Throckmorton St.), ...","[{'label': 'display', 'lat': 32.81077020211321...",32.810770,-96.811049,NaN,75219,TX,4b1a8a2af964a520e0eb23e3


In [22]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker([latitude,longitude], rdius=10,color='red',popup='Dallas City Hall', fill=True, fill_color='red', fill_opacity=0.6).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker([lat, lng], radius=5, color='blue', popup=label, fill=True,fill_color='blue', fill_opacity=0.6).add_to(venues_map)

venues_map